In [220]:
import pandas as pd
import numpy as np
from math import exp, log, sqrt, pi
import matplotlib.pyplot as plt
from scipy.stats import norm
import time

In [235]:
def direction(dt):
    u = np.exp(sigma*sqrt(dt) - 0.5*sigma*sigma*dt)
    d = np.exp(-sigma*sqrt(dt) - 0.5*sigma*sigma*dt)
    return(u,d)


def Intrinsic(s,k,v1,v2,p,option):
    if option == 'call':
        v0 = max(0,s-k)
        v1 = p*v1+(1-p)*v2
    
    if option == 'put':
        v0 = max(0,k-s)
        v1 = p*v1+(1-p)*v2
        
    v = max(v0,v1)
    return v
        
def Tree(s0,k,m,u,d,p,option,call_layer,dividend):
    
    # Create a 2D array for st through all the path
    rows, cols = (m+1, m+1)
    
    st = [[0 for i in range(cols)] for j in range(rows)] 
    st[0][0]=s0
    for i in range(1,cols):
        
        # minus dividend
        if i in list(dividend.keys()):
            for r in range(i):
                st[r][i-1]=st[r][i-1]*dividend[i]
        else:
            for r in range(i):
                st[r][i-1]=st[r][i-1]
        
        
        st[0][i]=st[0][i-1]*u
        for j in range(1,i+1):
            #st[j][i] = s0*u**(i-j)*d**j
            st[j][i]=st[j-1][i-1]*d

    
    # Create a 2D array for vt through all the path
    vt = [[0 for i in range(cols)] for j in range(rows)] 
    
    # Calc option value for layer m-1:
    for j in range(m+1):
        vt[j][m]=Intrinsic(st[j][m],k,0,0,p,option)
    
    for i in range(m-1,-1,-1):
        if i in call_layer:
            for j in range(i+1):
                vt[j][i] = Intrinsic(st[j][i],k,vt[j][i+1],vt[j+1][i+1],p,option)
        else:
            for j in range(i+1):
                vt[j][i] = p*vt[j][i+1]+(1-p)*vt[j+1][i+1]         
         
    return vt[0][0]
    #return vt

In [280]:
s0=100
sigma = 0.5
dt = 0.01
u,d = direction(dt)
m = 100
p=0.5
k=100
option = "call"
#call_layer = []
call_layer = [3,4,8,68,93,98]
#dividend = {2:1,3:1}
dividend = {2:0.98,10:0.98,90:0.98}

In [281]:
Tree(s0,k,m,u,d,p,option,call_layer,dividend)
#print(pd.DataFrame(Tree(s0,k,m,u,d,p,option,call_layer,dividend)))

16.710121639179725